In [ ]:
%load_ext tensorboard

import os
import tensorflow as tf
from datetime import datetime
from object_detection import model_lib
from object_detection import model_hparams
from object_detection.protos import pipeline_pb2

In [ ]:
def tensorflow_shutup(verbose=False):
    """
    Make Tensorflow less verbose
    """
    try:
        # noinspection PyPackageRequirements
        if not verbose:
            #import os
            from tensorflow.compat.v1 import logging
            logging.set_verbosity(logging.ERROR)
            os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

        # Monkey patching deprecation utils to shut it up! Maybe good idea to disable this once after upgrade
        # noinspection PyUnusedLocal
        def deprecated(date, instructions, warn_once=True):
            def deprecated_wrapper(func):
                return func
            return deprecated_wrapper

        from tensorflow.python.util import deprecation
        deprecation.deprecated = deprecated

    except ImportError:
        pass

def find_latest_checkpoint(log_dir, modelId):
    from glob import glob

    log_dir = os.path.normpath(log_dir)
    runs = glob(f"{log_dir}/{modelId}*")

    if not runs:
        return None

    runs.sort()
    latest_run = runs[-1]

    checkpoint = tf.train.get_checkpoint_state(latest_run)
    return checkpoint.model_checkpoint_path if checkpoint else None

In [ ]:
MODEL = "mobilenet-oid"

OUT_DIR = f"{MODEL}"
PIPELINE_CONFIG = f"{MODEL}/pipeline.config"
NUM_TRAIN_STEPS = 10000
CHECKPOINT_STEPS = 1000
MAX_CHECKPOINTS = None

In [ ]:
tensorflow_shutup(True)

In [ ]:
timestamp = datetime.now().strftime("-%Y%m%d-%H%M%S-%f")
modelId = os.path.basename(os.path.normpath(OUT_DIR))
out_dir = OUT_DIR + timestamp
log_dir = OUT_DIR

In [ ]:
config_proto = tf.ConfigProto()
config_proto.gpu_options.allow_growth = True
config = tf.estimator.RunConfig(model_dir=out_dir, session_config=config_proto, log_step_count_steps=1,
                                save_checkpoints_steps=CHECKPOINT_STEPS, keep_checkpoint_max=MAX_CHECKPOINTS)

In [ ]:
pipeline_overrides = None
latest_checkpoint = find_latest_checkpoint(log_dir, modelId)
if latest_checkpoint:
    pipeline_overrides = pipeline_pb2.TrainEvalPipelineConfig()
    pipeline_overrides.train_config.fine_tune_checkpoint = latest_checkpoint

In [ ]:
train_and_eval_dict = model_lib.create_estimator_and_inputs(
    run_config=config,
    pipeline_config_path=PIPELINE_CONFIG,
    config_override=pipeline_overrides,
    train_steps=NUM_TRAIN_STEPS,
    sample_1_of_n_eval_examples=1,
    hparams=model_hparams.create_hparams(None),
    save_final_config=True)

estimator = train_and_eval_dict['estimator']
train_input_fn = train_and_eval_dict['train_input_fn']
eval_input_fns = train_and_eval_dict['eval_input_fns']
eval_on_train_input_fn = train_and_eval_dict['eval_on_train_input_fn']
predict_input_fn = train_and_eval_dict['predict_input_fn']
train_steps = train_and_eval_dict['train_steps']

train_spec, eval_specs = model_lib.create_train_and_eval_specs(
        train_input_fn,
        eval_input_fns,
        eval_on_train_input_fn,
        predict_input_fn,
        train_steps,
        eval_on_train_data=False)

In [ ]:
%tensorboard --logdir "$log_dir"

In [ ]:
tf.estimator.train_and_evaluate(estimator, train_spec, eval_specs[0])